# Veterinary Business Intelligence System - ETL Pipeline (Task 2)

This notebook implements the Data Requirement, System Design & Development phase of the project.
It covers:
1.  **Data Extraction**: Fetching data from FDA API, Dog API, and a supplementary CSV source.
2.  **Staging**: Loading raw data into a staging environment (Pandas DataFrames).
3.  **Transformation**: Cleaning, normalizing, and enriching the data.
4.  **Data Warehousing**: Designing and populating a Star Schema in a local SQLite database.
5.  **Validation**: Running queries to ensure data quality and business relevance.

In [1]:
# 1. Setup and Library Imports
import pandas as pd
import numpy as np
import requests
import sqlite3
import json
import os
from datetime import datetime, timedelta

# Configuration
DATA_DIR = '../data'
RAW_DIR = os.path.join(DATA_DIR, 'raw')
PROCESSED_DIR = os.path.join(DATA_DIR, 'processed')
DB_PATH = os.path.join(DATA_DIR, 'veterinary_dw.db')

# Ensure directories exist
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("Libraries imported and directories setup.")

Libraries imported and directories setup.


In [2]:
import time
# 2. Data Extraction: FDA Adverse Event API

def fetch_fda_data(max_records_to_fetch, limit=500, api_key=None):
    """
    Fetches adverse event data from the openFDA Animal & Veterinary API.
    Note: For a full historical load (15 years), we would need to implement pagination
    and loop through 'skip' parameters. For this demo, we fetch a sample.
    """
    base_url = "https://api.fda.gov/animalandveterinary/event.json"

    # Query for the last 15 years (approximate start date)
    start_date = (datetime.now() - timedelta(days=15*365)).strftime('%Y%m%d')
    end_date = datetime.now().strftime('%Y%m%d')

    params = {
        'search': f'original_receive_date:[{start_date} TO {end_date}]',
        'limit': limit,
        'sort': 'original_receive_date:desc'
    }

    if api_key:
        params['api_key'] = api_key

    all_results = []
    total_fetched = 0

    session = requests.Session()
    current_url = base_url

    use_params_flag = True # this is used only on the first request, after that we use the next link as it contains params

    print(f"Fetching data from FDA API from period {start_date} to {end_date}\nThis may take a while for full data.")
    while True:
        try:
            # If it's the first request, use params.
            # If it's a subsequent request (via Link header), params are already in the URL.
            if use_params_flag:
                response = session.get(current_url, params=params)
            else:
                response = session.get(current_url)
            response.raise_for_status()
            data = response.json()

            batch = data.get('results', [])
            if not batch:
                break

            all_results.extend(batch)
            total_fetched += len(batch)
            print(f"Fetched {len(batch)} records so far. Total fetched: {total_fetched}")

            # Check for the next page as the FDA gives a link header with the url for next page of results
            if "next" in response.links:
                current_url = response.links["next"]["url"]
                # Here sleep for a bit (I don't want to DDOS the FDA... That would be rude)
                use_params_flag = False
                time.sleep(1)

            else:
                print("No more pages to fetch.")
                break

        except requests.exceptions.RequestException as e:
            print(f"HTTP error occurred: {e}")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break
        if total_fetched >= max_records_to_fetch:
            print(f"Reached the limit of {max_records_to_fetch} records.")
            break

    return all_results
# Fetch sample data
# Optional: Add your API key here if you have one, e.g., api_key="YOUR_KEY"
fda_raw_data = fetch_fda_data(max_records_to_fetch=10000, limit=1000, api_key=None)

# Save raw data to JSON for staging
fda_raw_path = os.path.join(RAW_DIR, 'fda_adverse_events.json')
print(fda_raw_path)
with open(fda_raw_path, 'w') as f:
    json.dump(fda_raw_data, f)
print(f"Raw FDA data saved to {fda_raw_path}")

Fetching data from FDA API from period 20110107 to 20260103
This may take a while for full data.
Fetched 1000 records so far. Total fetched: 1000
Fetched 1000 records so far. Total fetched: 2000
Fetched 1000 records so far. Total fetched: 3000
Fetched 1000 records so far. Total fetched: 4000
Fetched 1000 records so far. Total fetched: 5000
Fetched 1000 records so far. Total fetched: 6000
Fetched 1000 records so far. Total fetched: 7000
Fetched 1000 records so far. Total fetched: 8000
Fetched 1000 records so far. Total fetched: 9000
Fetched 1000 records so far. Total fetched: 10000
Reached the limit of 10000 records.
../data/raw/fda_adverse_events.json
Raw FDA data saved to ../data/raw/fda_adverse_events.json


In [3]:
# 3. Data Extraction: Dog Breed API

def fetch_dog_breeds(api_key=None):
    """
    Fetches dog breed classifications from TheDogAPI.
    """
    url = "https://api.thedogapi.com/v1/breeds"
    headers = {'x-api-key': api_key} if api_key else {}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        breeds = response.json()
        print(f"Successfully fetched {len(breeds)} dog breeds.")
        return breeds
    except Exception as e:
        print(f"Error fetching dog breeds: {e}")
        return []

# Optional: Add your API key here if you have one
dog_breeds_raw = fetch_dog_breeds(api_key=None)

# Save raw data
dog_raw_path = os.path.join(RAW_DIR, 'dog_breeds.json')
with open(dog_raw_path, 'w') as f:
    json.dump(dog_breeds_raw, f)
print(f"Raw Dog breed data saved to {dog_raw_path}")

Successfully fetched 170 dog breeds.
Raw Dog breed data saved to ../data/raw/dog_breeds.json


In [4]:
# 4. Data Extraction: Supplementary Breed Data (CSV)

# We are using a local CSV file for Cat Breeds as the supplementary source.
# This file was created in the 'data/raw' directory.

cat_breeds_path = os.path.join(RAW_DIR, 'cat_breeds.csv')

if os.path.exists(cat_breeds_path):
    print(f"Supplementary data found at {cat_breeds_path}")
else:
    print("Supplementary data file not found!")

Supplementary data file not found!


In [5]:
# 5. Staging Area: Raw Data Inspection and Schema Validation

# Load FDA Data
df_fda_staging = pd.DataFrame(fda_raw_data)
print("FDA Staging Data Shape:", df_fda_staging.shape)

# Load Dog Data
df_dog_staging = pd.DataFrame(dog_breeds_raw)
print("Dog Staging Data Shape:", df_dog_staging.shape)

# Load Cat Data
df_cat_staging = pd.read_csv(cat_breeds_path)
print("Cat Staging Data Shape:", df_cat_staging.shape)

# Basic Schema Validation
required_fda_columns = ['original_receive_date', 'animal', 'drug', 'reaction']
missing_cols = [col for col in required_fda_columns if col not in df_fda_staging.columns]
if missing_cols:
    print(f"Warning: Missing columns in FDA data: {missing_cols}")
else:
    print("FDA Schema Validation Passed.")

df_fda_staging.head(2)

FDA Staging Data Shape: (10000, 19)
Dog Staging Data Shape: (170, 13)


FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/cat_breeds.csv'

In [ ]:
# 6. Transformation: Cleaning FDA Event Data

def clean_fda_data(df):
    # 1. Flatten 'animal' column
    # The 'animal' column contains a dictionary (gender, breed, weight, etc.)
    if 'animal' in df.columns:
        animal_df = pd.json_normalize(df['animal'])
        # Rename columns to avoid collision if needed, or just merge
        df = pd.concat([df.drop(['animal'], axis=1), animal_df], axis=1)

    # 2. Flatten 'receiver' column for Geography
    if 'receiver' in df.columns:
        receiver_df = pd.json_normalize(df['receiver'])
        # Keep only relevant location columns
        loc_cols = ['city', 'state', 'country', 'postal_code']
        for col in loc_cols:
            if col in receiver_df.columns:
                df[f'receiver_{col}'] = receiver_df[col]
            else:
                df[f'receiver_{col}'] = 'Unknown'
        df.drop(['receiver'], axis=1, inplace=True, errors='ignore')

    # 3. Handle Missing Values (MSK)
    # Replace 'MSK' with NaN or a placeholder
    df.replace('MSK', np.nan, inplace=True)

    # 4. Standardize Dates
    df['event_date'] = pd.to_datetime(df['original_receive_date'], format='%Y%m%d', errors='coerce')

    # 5. Standardize Gender
    if 'gender' in df.columns:
        df['gender'] = df['gender'].fillna('Unknown').str.title()

    # 6. Standardize Reproductive Status
    if 'reproductive_status' in df.columns:
        df['reproductive_status'] = df['reproductive_status'].fillna('Unknown').str.title()

    return df

df_fda_cleaned = clean_fda_data(df_fda_staging.copy())
print("FDA Data Cleaned.")
# Check for new geography columns
print(df_fda_cleaned[['event_date', 'gender', 'receiver_country', 'receiver_state']].head())

FDA Data Cleaned.
  event_date   gender receiver_country receiver_state
0 2011-10-05     Male              USA             MD
1 2011-04-20   Female              USA             MD
2 2011-03-17     Male              USA             MD
3 2011-07-21  Unknown              USA             MD
4 2015-06-29   Female              USA             MD


In [ ]:
# 7. Transformation: Normalizing Active Ingredients

def normalize_drugs(df):
    """
    Explodes the 'drug' list and extracts active ingredients.
    """
    # Explode the list of drugs so each drug gets a row
    df_exploded = df.explode('drug')

    # Normalize the drug dictionary
    drug_details = pd.json_normalize(df_exploded['drug'])

    # We need to keep the original index to merge back or just concat if indices align
    # Reset index of exploded df to align with drug_details
    df_exploded = df_exploded.reset_index(drop=True)
    drug_details = drug_details.reset_index(drop=True)

    # Combine
    df_combined = pd.concat([df_exploded.drop(['drug'], axis=1), drug_details], axis=1)

    # Extract Active Ingredients
    # The API structure for active ingredients can be complex.
    # Often it's a list of dicts inside the drug dict.
    # For simplicity, we will try to extract 'active_ingredients' list and join them string

    def extract_ingredients(ingredients_list):
        if isinstance(ingredients_list, list):
            names = [item.get('name', '') for item in ingredients_list if 'name' in item]
            return ", ".join(names)
        return "Unknown"

    if 'active_ingredients' in df_combined.columns:
        df_combined['active_ingredient_normalized'] = df_combined['active_ingredients'].apply(extract_ingredients)
    else:
        df_combined['active_ingredient_normalized'] = "Unknown"

    return df_combined

df_fda_drugs = normalize_drugs(df_fda_cleaned)
print("Drugs Normalized. New Shape:", df_fda_drugs.shape)
df_fda_drugs[['brand_name', 'active_ingredient_normalized']].head()

Drugs Normalized. New Shape: (786, 64)


,brand_name,active_ingredient_normalized
0,MSK,"Milbemycin, Lufenuron"
1,MSK,
2,MSK,
3,MSK,
4,MSK,Carprofen


In [ ]:
# 8. Transformation: Enriching with Breed Groups and Purpose

# Prepare Dog Breed Data for Merge
# We want to map 'breed.breed_component' from FDA to 'name' in Dog API to get 'breed_group' and 'bred_for'
df_dog_lookup = df_dog_staging[['name', 'breed_group', 'bred_for']].copy()
df_dog_lookup['name'] = df_dog_lookup['name'].str.lower()

# Prepare FDA Data
# The breed information is in 'breed.breed_component' (which might be a string or list)
# After flattening in step 6, it might be 'breed.breed_component'
# Let's inspect columns to find breed info
breed_col = [c for c in df_fda_drugs.columns if 'breed' in c.lower()]
print("Breed columns found:", breed_col)

# Assuming 'breed.breed_component' is the column and it might be a string
if 'breed.breed_component' in df_fda_drugs.columns:
    df_fda_drugs['breed_lower'] = df_fda_drugs['breed.breed_component'].astype(str).str.lower()

    # Merge
    df_enriched = pd.merge(
        df_fda_drugs,
        df_dog_lookup,
        left_on='breed_lower',
        right_on='name',
        how='left'
    )

    # Fill NaN for non-dogs or unmatched breeds
    df_enriched['breed_group'] = df_enriched['breed_group'].fillna('Unknown')
    df_enriched['bred_for'] = df_enriched['bred_for'].fillna('Unknown')
else:
    print("Breed column not found as expected. Skipping enrichment.")
    df_enriched = df_fda_drugs
    df_enriched['breed_group'] = 'Unknown'
    df_enriched['bred_for'] = 'Unknown'

print("Data Enriched.")
df_enriched[['breed.breed_component', 'breed_group', 'bred_for']].head()

Breed columns found: ['breed.is_crossbred', 'breed.breed_component']
Data Enriched.


,breed.breed_component,breed_group,bred_for
0,Retriever - Labrador,Unknown,Unknown
1,Retriever - Labrador,Unknown,Unknown
2,Retriever - Labrador,Unknown,Unknown
3,Retriever - Labrador,Unknown,Unknown
4,Retriever - Labrador,Unknown,Unknown


In [ ]:
# 9. Data Warehouse Schema Design (Advanced Star Schema)

# Connect to SQLite Database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Drop tables if they exist
cursor.executescript("""
DROP TABLE IF EXISTS FactAdverseEvents;
DROP TABLE IF EXISTS DimTime;
DROP TABLE IF EXISTS DimAnimal;
DROP TABLE IF EXISTS DimDrug;
DROP TABLE IF EXISTS DimReaction;
DROP TABLE IF EXISTS DimGeography;
DROP TABLE IF EXISTS DimOutcome;
""")

# Create Advanced Dimension Tables
create_dims_sql = """
-- Enhanced Time Dimension
CREATE TABLE DimTime (
    TimeKey INTEGER PRIMARY KEY AUTOINCREMENT,
    Date DATE UNIQUE,
    Year INTEGER,
    Month INTEGER,
    Day INTEGER,
    Quarter INTEGER,
    DayOfWeek TEXT,
    IsWeekend BOOLEAN
);

-- Geography Dimension
CREATE TABLE DimGeography (
    GeographyKey INTEGER PRIMARY KEY AUTOINCREMENT,
    City TEXT,
    State TEXT,
    Country TEXT,
    PostalCode TEXT
);

-- Outcome Dimension
CREATE TABLE DimOutcome (
    OutcomeKey INTEGER PRIMARY KEY AUTOINCREMENT,
    OutcomeName TEXT UNIQUE,
    SeverityLevel TEXT
);

-- Animal Dimension
CREATE TABLE DimAnimal (
    AnimalKey INTEGER PRIMARY KEY AUTOINCREMENT,
    Species TEXT,
    Breed TEXT,
    Gender TEXT,
    ReproductiveStatus TEXT,
    BreedGroup TEXT,
    BreedPurpose TEXT
);

-- Drug Dimension
CREATE TABLE DimDrug (
    DrugKey INTEGER PRIMARY KEY AUTOINCREMENT,
    BrandName TEXT,
    ActiveIngredients TEXT,
    Manufacturer TEXT
);

-- Reaction Dimension
CREATE TABLE DimReaction (
    ReactionKey INTEGER PRIMARY KEY AUTOINCREMENT,
    ReactionName TEXT UNIQUE
);
"""

# Create Fact Table with Foreign Keys and Indexes
# Added DaysToOnset to cover "How many days it takes for the reactions to appear"
create_fact_sql = """
CREATE TABLE FactAdverseEvents (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    FDA_ReportID TEXT,
    TimeKey INTEGER,
    AnimalKey INTEGER,
    DrugKey INTEGER,
    ReactionKey INTEGER,
    GeographyKey INTEGER,
    OutcomeKey INTEGER,
    Age FLOAT,
    Weight FLOAT,
    DaysToOnset INTEGER, -- New field for reaction latency analysis
    -- Measures
    ReactionCount INTEGER DEFAULT 1,
    FOREIGN KEY(TimeKey) REFERENCES DimTime(TimeKey),
    FOREIGN KEY(AnimalKey) REFERENCES DimAnimal(AnimalKey),
    FOREIGN KEY(DrugKey) REFERENCES DimDrug(DrugKey),
    FOREIGN KEY(ReactionKey) REFERENCES DimReaction(ReactionKey),
    FOREIGN KEY(GeographyKey) REFERENCES DimGeography(GeographyKey),
    FOREIGN KEY(OutcomeKey) REFERENCES DimOutcome(OutcomeKey)
);

-- Create Indexes for Performance
CREATE INDEX idx_fact_time ON FactAdverseEvents(TimeKey);
CREATE INDEX idx_fact_animal ON FactAdverseEvents(AnimalKey);
CREATE INDEX idx_fact_drug ON FactAdverseEvents(DrugKey);
CREATE INDEX idx_fact_geo ON FactAdverseEvents(GeographyKey);
"""

cursor.executescript(create_dims_sql)
cursor.executescript(create_fact_sql)
conn.commit()
print("Advanced Data Warehouse Schema Created with Indexes and New Dimensions.")

Advanced Data Warehouse Schema Created with Indexes and New Dimensions.


In [ ]:
# FIX: Populate the Time Dimension before loading facts
from datetime import datetime, timedelta

def populate_time_dimension(start_year=2000, end_year=2025):
    print("Populating DimTime...")

    # Connect (ensure we use the same connection object name as your script)
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    batch_data = []
    while current_date <= end_date:
        is_weekend = current_date.weekday() >= 5 # 5=Sat, 6=Sun
        batch_data.append((
            str(current_date.date()), # Date string
            current_date.year,
            current_date.month,
            current_date.day,
            (current_date.month - 1) // 3 + 1, # Quarter
            current_date.strftime('%A'), # DayOfWeek
            is_weekend
        ))
        current_date += timedelta(days=1)

    # Bulk Insert
    cursor.executemany("""
        INSERT OR IGNORE INTO DimTime
        (Date, Year, Month, Day, Quarter, DayOfWeek, IsWeekend)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, batch_data)

    conn.commit()
    conn.close()
    print("✅ DimTime populated successfully.")

# Run the fix
populate_time_dimension()

In [ ]:
# 10. Load: Populating Dimension Tables

def get_or_create_key(table, search_cols, values, key_col):
    where_clause = " AND ".join([f"{col} = ?" for col in search_cols])
    query = f"SELECT {key_col} FROM {table} WHERE {where_clause}"
    cursor.execute(query, values)
    result = cursor.fetchone()

    if result:
        return result[0]
    else:
        cols = ", ".join(search_cols)
        placeholders = ", ".join(["?" for _ in search_cols])
        insert_sql = f"INSERT INTO {table} ({cols}) VALUES ({placeholders})"
        cursor.execute(insert_sql, values)
        return cursor.lastrowid

# 1. Populate DimTime (Enhanced)
unique_dates = df_enriched['event_date'].dropna().unique()
for date_val in unique_dates:
    dt = pd.to_datetime(date_val)
    day_name = dt.day_name()
    is_weekend = 1 if dt.weekday() >= 5 else 0

    cursor.execute("""
        INSERT OR IGNORE INTO DimTime (Date, Year, Month, Day, Quarter, DayOfWeek, IsWeekend)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (dt.date(), dt.year, dt.month, dt.day, dt.quarter, day_name, is_weekend))
conn.commit()
print("DimTime Populated.")

# 2. Populate DimReaction
if 'reaction' in df_enriched.columns:
    all_reactions = set()
    for reactions in df_enriched['reaction']:
        if isinstance(reactions, list):
            for r in reactions:
                if 'name' in r:
                    all_reactions.add(r['name'])

    for r_name in all_reactions:
        cursor.execute("INSERT OR IGNORE INTO DimReaction (ReactionName) VALUES (?)", (r_name,))
    conn.commit()
print("DimReaction Populated.")

# 3. Populate DimOutcome (New)
# Extract unique outcomes from the 'outcome' column (which might be a list or dict in raw data)
# In flattened data, it might be 'outcome.medical_status' or just 'outcome'
# Let's inspect the column in the loop or assume it's 'outcome'
# For now, we'll populate it dynamically in the Fact Load loop or pre-scan here.
# Let's pre-scan if possible, or just use get_or_create_key in the loop.
print("DimOutcome will be populated dynamically.")

# 4. Populate DimGeography (New)
# Similarly, we can populate dynamically or pre-scan.
print("DimGeography will be populated dynamically.")

DimTime Populated.
DimReaction Populated.
DimOutcome will be populated dynamically.
DimGeography will be populated dynamically.


In [ ]:
# 11. Load: Populating Fact Table (Advanced)

print("Loading Fact Table... this may take a moment.")

count = 0
for index, row in df_enriched.iterrows():
    try:
        # 1. Time Key
        event_date = row['event_date']
        if pd.isnull(event_date):
            continue
        dt = pd.to_datetime(event_date)
        cursor.execute("SELECT TimeKey FROM DimTime WHERE Date = ?", (str(dt.date()),))
        time_res = cursor.fetchone()
        time_key = time_res[0] if time_res else None

        # 2. Animal Key
        species = row.get('species', 'Unknown')
        breed = row.get('breed.breed_component', 'Unknown')
        gender = row.get('gender', 'Unknown')
        repro_status = row.get('reproductive_status', 'Unknown')
        breed_group = row.get('breed_group', 'Unknown')
        breed_purpose = row.get('bred_for', 'Unknown')

        animal_key = get_or_create_key(
            'DimAnimal',
            ['Species', 'Breed', 'Gender', 'ReproductiveStatus', 'BreedGroup', 'BreedPurpose'],
            (str(species), str(breed), str(gender), str(repro_status), str(breed_group), str(breed_purpose)),
            'AnimalKey'
        )

        # 3. Drug Key
        brand = row.get('brand_name', 'Unknown')
        active_ing = row.get('active_ingredient_normalized', 'Unknown')
        manufacturer = row.get('manufacturer.name', 'Unknown')

        drug_key = get_or_create_key(
            'DimDrug',
            ['BrandName', 'ActiveIngredients', 'Manufacturer'],
            (str(brand), str(active_ing), str(manufacturer)),
            'DrugKey'
        )

        # 4. Geography Key
        city = row.get('receiver_city', 'Unknown')
        state = row.get('receiver_state', 'Unknown')
        country = row.get('receiver_country', 'Unknown')
        postal = row.get('receiver_postal_code', 'Unknown')

        geo_key = get_or_create_key(
            'DimGeography',
            ['City', 'State', 'Country', 'PostalCode'],
            (str(city), str(state), str(country), str(postal)),
            'GeographyKey'
        )

        # 5. Outcome Key
        outcome_raw = row.get('outcome', [])
        outcome_name = "Unknown"
        if isinstance(outcome_raw, list) and len(outcome_raw) > 0:
             outcome_name = outcome_raw[0].get('medical_status', 'Unknown')
        elif isinstance(outcome_raw, str):
            outcome_name = outcome_raw

        severity = "Normal"
        if outcome_name.lower() in ['died', 'euthanized']:
            severity = "Critical"

        outcome_key = get_or_create_key(
            'DimOutcome',
            ['OutcomeName', 'SeverityLevel'],
            (str(outcome_name), severity),
            'OutcomeKey'
        )

        # 6. Reaction Key & Fact Insertion
        reactions_list = row.get('reaction', [])
        if isinstance(reactions_list, list):
            for r in reactions_list:
                r_name = r.get('name', 'Unknown')

                # Get Reaction Key
                cursor.execute("SELECT ReactionKey FROM DimReaction WHERE ReactionName = ?", (r_name,))
                res = cursor.fetchone()
                if not res:
                    cursor.execute("INSERT INTO DimReaction (ReactionName) VALUES (?)", (r_name,))
                    reaction_key = cursor.lastrowid
                else:
                    reaction_key = res[0]

                # Insert Fact
                fda_id = row.get('report_id', 'Unknown')
                age = row.get('age.min', 0)
                weight = row.get('weight.min', 0)

                # Calculate DaysToOnset (if onset_date exists)
                # Note: This field might not exist in all API responses, so we handle gracefully
                days_to_onset = None
                if 'onset_date' in row and pd.notnull(row['onset_date']):
                    try:
                        onset_dt = pd.to_datetime(row['onset_date'], format='%Y%m%d', errors='coerce')
                        if pd.notnull(onset_dt):
                            days_to_onset = (onset_dt - dt).days
                    except:
                        pass

                cursor.execute("""
                    INSERT INTO FactAdverseEvents
                    (FDA_ReportID, TimeKey, AnimalKey, DrugKey, ReactionKey, GeographyKey, OutcomeKey, Age, Weight, DaysToOnset)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (str(fda_id), time_key, animal_key, drug_key, reaction_key, geo_key, outcome_key, age, weight, days_to_onset))

        count += 1
        if count % 100 == 0:
            conn.commit()

    except Exception as e:
        continue

conn.commit()
print(f"Finished loading Advanced Fact Table. Processed {count} events.")

Loading Fact Table... this may take a moment.
Finished loading Advanced Fact Table. Processed 786 events.


In [ ]:
# 12. Data Quality Checks and Validation (Advanced Queries)

# Query 1: Geographic Distribution of Events
print("--- Analysis: Geographic Distribution (Top Countries) ---")
query_geo = """
SELECT
    g.Country,
    COUNT(f.EventID) as EventCount
FROM FactAdverseEvents f
JOIN DimGeography g ON f.GeographyKey = g.GeographyKey
GROUP BY g.Country
ORDER BY EventCount DESC
LIMIT 10;
"""
print(pd.read_sql_query(query_geo, conn))

# Query 2: Critical Outcomes by Breed Group
print("\n--- Analysis: Critical Outcomes by Breed Group ---")
query_outcome = """
SELECT
    a.BreedGroup,
    COUNT(f.EventID) as CriticalCount
FROM FactAdverseEvents f
JOIN DimAnimal a ON f.AnimalKey = a.AnimalKey
JOIN DimOutcome o ON f.OutcomeKey = o.OutcomeKey
WHERE o.SeverityLevel = 'Critical'
GROUP BY a.BreedGroup
ORDER BY CriticalCount DESC;
"""
print(pd.read_sql_query(query_outcome, conn))

# Query 3: Seasonal Analysis (Events by Month)
print("\n--- Analysis: Seasonality (Events by Month) ---")
query_season = """
SELECT
    t.Month,
    COUNT(f.EventID) as Count
FROM FactAdverseEvents f
JOIN DimTime t ON f.TimeKey = t.TimeKey
GROUP BY t.Month
ORDER BY t.Month;
"""
print(pd.read_sql_query(query_season, conn))

conn.close()

--- Analysis: Geographic Distribution (Top Countries) ---
  Country  EventCount
0     USA        2461

--- Analysis: Critical Outcomes by Breed Group ---
  BreedGroup  CriticalCount
0    Unknown            719
1        Toy             20
2      Hound             16

--- Analysis: Seasonality (Events by Month) ---
    Month  Count
0       1     78
1       2     83
2       3    233
3       4    281
4       5    293
5       6    111
6       7    308
7       8    196
8       9    234
9      10    189
10     11    340
11     12    115
